In [1]:
using BenchmarkTools, Compat, DataFrames, Distributions, ForwardDiff

In [2]:
# Newton, mixed logit.

df = readtable("../data/parsed_model_australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b
const n_simulations = 5
const simulated_b = 5

srand(123456)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

rand_contdist (generic function with 1 method)

In [3]:
mixed_logit = DataFrame(ID = 1.0:Float64(n_individuals),
                        β2 = 1.0:Float64(n_individuals),
                        β3 = 1.0:Float64(n_individuals),
                        β4 = 1.0:Float64(n_individuals),
                        β5 = 1.0:Float64(n_individuals))

function simulate()
    for i = 1:n_individuals, j = 1:n_simulations
        mixed_logit[i, j] = rand()
    end
end

simulate()

head(mixed_logit)

,ID,β2,β3,β4,β5
1,0.5152104671804307,0.7412898556237781,0.5821424283521601,0.2099687343318144,0.1806485932541042
2,0.5743318350142363,0.4733804534506725,0.6350533219682404,0.003946023046596414,0.9987169142802474
3,0.391493560981647,0.011082822655117397,0.728900346094737,0.5885694843677474,0.0955464156850605
4,0.3161831307020315,0.9868070201531223,0.3248312054217537,0.6275088202683465,0.9742043509623348
5,0.9133245457961849,0.3026406935881607,0.24034692925733436,0.6929933674978992,0.9948456891059492
6,0.7798750382138562,0.7603354924796097,0.9018641475932665,0.09221139730637495,0.400661483972383


In [4]:
function individual(θ::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function construct(γ::Vector, θ::Vector)
        return θ[1]+θ[2]*γ[1]
    end
    
    function probability(θ::Vector)
        logit = 0.0
        t = 0.0
        for k = 1:n_simulations
            β = []
            for j = 1:simulated_b-1
                push!(β, θ[j])
            end
            γ = [mixed_logit[i, k]]
            push!(β, construct(γ, θ[simulated_b:simulated_b+1]))
            for j = simulated_b+2:length(θ)
                push!(β, θ[j])
            end 
            c = utility(β, choice)
            for alternative in alternatives
                t += exp(utility(β, alternative)-c)
            end
            logit += 1/(1+t)
        end
        return logit/n_simulations
    end
    
    return probability
end

individual (generic function with 1 method)

In [5]:
function f(θ::Vector)
    model = 0.0
    i::Int64 = 1
    while i <= n_individuals
        probability = individual(θ, i)
        model += log(probability(θ))
        i += 1
    end
    return -model/n_individuals
end

f (generic function with 1 method)

In [6]:
function g(x::Vector)
    t = zeros(length(x))
    for i = 1:n_individuals
        probability = individual(x, i)
        t += (1/probability(x))*ForwardDiff.gradient(probability, x)
    end
    return -t/n_individuals
end

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [7]:
function H(x::Vector)
    return ForwardDiff.hessian(f, x)
end

function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [8]:
function BHHH(x::Vector)
    t = zeros(length(x), length(x))
    for i = 1:n_individuals
        probability = individual(x, i)
        g = ForwardDiff.gradient(probability, x)
        t += g*(g')
    end
    return t/n_individuals
end

function BHHH!(x::Vector, storage::Matrix)
    s = BHHH(x)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

BHHH! (generic function with 1 method)

In [9]:
function BFGS(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B-(Bs*Bs')/dot(s, Bs)+(y*y')/dot(s, y)
end

function BFGS!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = BFGS(B, y, s)
end

BFGS! (generic function with 1 method)

In [10]:
function SR1(B::Matrix, y::Vector, s::Vector)
    Bs = B*s
    return B+((y-Bs)*(y-Bs)')/((y-Bs)'*s)
end

function SR1!(B::Matrix, y::Vector, s::Vector)
    n, m = size(B)
    B[1:n, 1:m] = SR1(B, y, s)
end

SR1! (generic function with 1 method)

In [11]:
function newton(f::Function, g::Function, h::Function, x0::Vector,
        δ::Float64 = 1e-6, nmax::Int64 = 100)
    k = 1
    x = x0
    n = length(x)
    δ *=δ
    H = eye(n)
    dfx = ones(n)
    g(x, dfx)
    while dot(dfx, dfx) > δ && k <= nmax
        k += 1
        g(x, dfx)
        h(x, H)
        x -= H\dfx
    end
    return x, k
end

newton (generic function with 3 methods)

In [12]:
newton(f, g!, H!, zeros(7))

([7.36212, 5.22501, 4.45487, 0.0108628, -0.135201, 0.00712103, -0.018467], 7)